# What fraction of the basic reproduction number "R" of COVID-19 in The Netherlands is explained by the number of positive tests in the municipality of Amsterdam?
## A study of correlation using daily Dutch COVID-19 data of the National Institute for Public Health and the Environment (RIVM)

Used data: 
- https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/ed0699d1-c9d5-4436-8517-27eb993eab6e
- https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/5f6bc429-1596-490e-8618-1ed8fd768427

In [1]:
from datetime import datetime
from IPython.display import display
import numpy as np
import pandas as pd
from pandas.tseries.frequencies import to_offset
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Span, Label, ColumnDataSource, Arrow, NormalHead
output_notebook()

COLORS=('#7570b3', '#d95f02', '#1b9e77')

Loading BokehJS ...

In [2]:
def load_data():
    """
    Author: Tijs van Lieshout
    Loading of both datasets, outputting some stats about the datasets and preparing them for further use

    Keyword arguments:
    None
    
    Returns:
    cases_per_municipality_per_day -- Dataframe containing COVID-19 cases per municipality per day for the Netherlands
    amsterdam_cases_per_day -- Dataframe containing COVID-19 cases in Amsterdam per day for the Netherlands
    total_cases_without_amsterdam_per_day -- Dataframe containing COVID-19 cases per municipality per day 
    for the Netherlands not including Amsterdam
    R_per_day -- Dataframe containing reproduction number R per (available) day for the Netherlands
    """
    # Loading and describing the RIVM data "cases per municipality per day"
    cases_per_municipality_per_day = pd.read_csv("COVID-19_aantallen_gemeente_per_dag.csv", sep=";")
    cases_per_municipality_per_day['Date_of_publication'] = pd.to_datetime(cases_per_municipality_per_day['Date_of_publication'], 
                                                                      format='%Y-%m-%d')
    cases_per_municipality_per_day = cases_per_municipality_per_day.set_index('Date_of_publication').sort_index()
    print(f"Total number of COVID-19 cases: {cases_per_municipality_per_day.Total_reported.sum()}")
    print(f"Total number of COVID-19 hospital admissions: {cases_per_municipality_per_day.Hospital_admission.sum()}")
    print(f"Total number of COVID-19 deaths: {cases_per_municipality_per_day.Deceased.sum()}")
    print(f"Number of rows in cases_per_municipality_per_day: {len(cases_per_municipality_per_day.index)}")

    # Creating a subset of only statistics about the municipality of Amsterdam
    amsterdam_cases_per_day = cases_per_municipality_per_day[cases_per_municipality_per_day.Municipality_name == 'Amsterdam']
    total_cases_without_amsterdam_per_day = cases_per_municipality_per_day[cases_per_municipality_per_day.Municipality_name != 'Amsterdam']
    # display(amsterdam_cases_per_day)
    print(f"Number of rows in amsterdam_cases_per_day: {len(amsterdam_cases_per_day.index)}")

    # Loading and describing the RIVM data "COVID-19 reproduction number"
    R_per_day = pd.read_json('COVID-19_reproductiegetal.json')
    R_per_day = R_per_day.set_index('Date').sort_index()
    # display(R_per_day)
    return cases_per_municipality_per_day, amsterdam_cases_per_day, total_cases_without_amsterdam_per_day, R_per_day

In [3]:
cases_per_municipality_per_day, amsterdam_cases_per_day, total_cases_without_amsterdam_per_day, R_per_day = load_data()

Total number of COVID-19 cases: 1051965
Total number of COVID-19 hospital admissions: 23761
Total number of COVID-19 deaths: 15200
Number of rows in cases_per_municipality_per_day: 136080
Number of rows in amsterdam_cases_per_day: 720


In [4]:
 def plot_reproduction_number(R_per_day):
    """
    Author: Tijs van Lieshout
    Plotting the reproduction number R for the Netherlands

    Keyword arguments:
    R_per_day -- Dataframe containing reproduction number R per (available) day for the Netherlands
    
    Returns:
    None
    """
    p = figure(plot_width=1000, plot_height=800, x_axis_type="datetime",
               title="Reproduction number 'R' per day in the Netherlands",
               x_axis_label="Date", y_axis_label="Reproduction number", 
               toolbar_location=None, tools="",
               y_range=(0, 5))

    p.varea(x=R_per_day.index,
            y1=R_per_day.Rt_low,
            y2=R_per_day.Rt_up, color='#BDBDBD', legend_label="95% Confidence band")

    p.line(R_per_day.index, R_per_day.Rt_avg, color=COLORS[1], 
           legend_label="R average")

    hline = Span(location=1, dimension='width', line_color='grey', 
                 line_width=1, line_dash='dashed')
    p.add_layout(hline)

    add_calculation_R_change_annotation(p)

    p.x_range.start = R_per_day.index.min()
    p.x_range.end = R_per_day.index.max()
    p.legend.location = 'top_right'

    show(p)

In [5]:
def add_calculation_R_change_annotation(p):
    """
    Author: Tijs van Lieshout
    Adding annotation about the switch of methods from the RIVM at 2020-06-12 
    for the reproduction number R of COVID-19 for the Netherlands plot

    Keyword arguments:
    p -- Bokeh figure of the reproduction number R plot
    
    Returns:
    None
    """
    switch_in_methods_date = datetime.strptime('2020-06-12', '%Y-%m-%d').date()
    vline = Span(location=switch_in_methods_date, dimension='height', line_color=COLORS[0], 
                 line_width=1, line_dash='dashed')
    p.add_layout(vline)
    label1 = Label(x=switch_in_methods_date, y=4.8, x_offset = 2,
                  text='Day the RIVM switched the calculation of R', text_font_size='14px')
    label2 = Label(x=switch_in_methods_date, y=4.7, x_offset = 2,
                  text='from based on hospitalisations to positive tests', text_font_size='14px')
    p.add_layout(label1)
    p.add_layout(label2)

In [6]:
plot_reproduction_number(R_per_day)

## Figure 1: Reproduction number 'R' per day in the Netherlands
#### FIGURE CAPTION INFORMATION GOES HERE (TODO)

In [7]:
def plot_cases_amsterdam_vs_other(total_cases_without_amsterdam, amsterdam_cases, reproduction_numbers):
    max_y_range = total_cases_without_amsterdam.Total_reported.max() + total_cases_without_amsterdam.Total_reported.max()/10
    
    p = figure(plot_width=1000, plot_height=800, x_axis_type="datetime",
               title=f"Total number of new COVID-19 cases per day in the Amsterdam vs. other municipalities in the Netherlands",
               x_axis_label="Date", y_axis_label="Total new COVID-19 cases", 
               toolbar_location=None, tools="", y_range=(0, max_y_range))
    
    p.circle(x=total_cases_without_amsterdam.index, 
             y=total_cases_without_amsterdam.Total_reported,
             legend_label='New COVID-19 cases in another municipality per day',
             color="#BDBDBD")
    
    p.circle(x=amsterdam_cases.index, 
             y=amsterdam_cases.Total_reported,
             legend_label='New COVID-19 cases in Amsterdam per day',
             color=COLORS[1])
    
    p.x_range.start = total_cases_without_amsterdam.index.min()
    p.x_range.end = total_cases_without_amsterdam.index.max()
    p.legend.location = 'top_left'
    
    show(p)

In [8]:
plot_cases_amsterdam_vs_other(total_cases_without_amsterdam_per_day, amsterdam_cases_per_day, R_per_day)

## Figure 2: Total number of new COVID-19 cases per day in the Amsterdam vs. other municipalities in the Netherlands
#### TODO ADD CAPTION

In [9]:
def plot_cases_netherlands_amsterdam(total_cases_without_amsterdam, amsterdam_cases, y_label_pos, timeperiod):
    """
    Author: Tijs van Lieshout
    Plotting the COVID-19 cases for the Netherlands and Amsterdam

    Keyword arguments:
    total_cases_without_amsterdam -- Dataframe containing COVID-19 cases per municipality 
    for the Netherlands not including Amsterdam
    amsterdam_cases -- Dataframe containing COVID-19 cases in Amsterdam for the Netherlands
    y_label_pos -- Int for the position of the annotation in the plot
    timeperiod -- String to put in the title of the plot (e.g. 'week' or 'day')
    
    Returns:
    None
    """
    total_cases = total_cases_without_amsterdam.groupby(total_cases_without_amsterdam.index).sum()
    amsterdam_total_cases = amsterdam_cases.groupby(amsterdam_cases.index).sum()
    
    source = ColumnDataSource(data=dict(
    x=total_cases.index,
    y1=amsterdam_total_cases.Total_reported,
    y2=total_cases.Total_reported,
    ))
    
    max_y_range = total_cases.Total_reported.max() + total_cases.Total_reported.max()/10
    
    p = figure(plot_width=1000, plot_height=800, x_axis_type="datetime",
               title=f"Total number of new COVID-19 cases per {timeperiod} in the Netherlands",
               x_axis_label="Date", y_axis_label="Total new COVID-19 cases", 
               toolbar_location=None, tools="", y_range=(0, max_y_range))

    p.varea_stack(['y1', 'y2'], x='x', color=("grey", "#BDBDBD"),
                  legend_label=("Amsterdam", "The Netherlands"), source=source)

    # Data from CBS 2020
    population_of_amsterdam = 872779
    population_of_the_netherlands = 17407585
    
    p.line(x=total_cases.index, 
           y=total_cases.Total_reported*(population_of_amsterdam/population_of_the_netherlands), 
           color=COLORS[1], line_width=2, 
           legend_label='Expected fraction of Dutch cases to be in Amsterdam based on the population')
    
    add_COVID_distribution_annotation(p, total_cases, y_label_pos)
    
    p.x_range.start = total_cases.index.min()
    p.x_range.end = total_cases.index.max()
    p.legend.location = 'top_left'

    show(p)

In [10]:
def add_COVID_distribution_annotation(p, total_cases, y_label_pos):
    """
    Author: Tijs van Lieshout
    Plotting the COVID-19 cases for the Netherlands and Amsterdam

    Keyword arguments:
    p -- Bokeh figure of the reproduction number R plot
    total_cases -- Dataframe containing COVID-19 cases per municipality 
    for the Netherlands not including Amsterdam
    y_label_pos -- Int for the position of the annotation in the plot
    
    Returns:
    None
    """
    highlighted_date_start = datetime.strptime('2020-09-01', '%Y-%m-%d').date()
    highlighted_date_end = datetime.strptime('2020-12-01', '%Y-%m-%d').date()
    
    label1 = Label(x=highlighted_date_start, y=y_label_pos+y_label_pos/10, x_offset = -400, y_offset=0, text_color='black',
                  text='If COVID-19 cases are uniformly distributed over the Netherlands and Amsterdam', text_font_size='14px')
    label2 = Label(x=highlighted_date_start, y=y_label_pos, x_offset = -400, y_offset=0, text_color='black',
                  text='you would expect the orange line to follow the dark grey area', text_font_size='14px')
    
    arrow = Arrow(end=NormalHead(line_color="black", line_width=1), x_start=highlighted_date_start, 
                  y_start=y_label_pos, x_end=highlighted_date_end, y_end=y_label_pos/6)
    
    p.add_layout(label1)
    p.add_layout(label2)
    p.add_layout(arrow)

In [11]:
plot_cases_netherlands_amsterdam(total_cases_without_amsterdam_per_day, amsterdam_cases_per_day, 3000, 'day')

## Figure 3: Total number of new COVID-19 cases per day in the Netherlands and Amsterdam
#### In darker grey you can see the number of positive COVID-19 cases in Amsterdam, in light-grey the number of positive COVID-19 cases in the Netherlands. In orange you can see the expected number of positive cases by simply dividing the population of amsterdam by the population of the netherlands and multiplying it with the number of positive cases in the Netherlands. You can see that in the beginning of the second wave there were more positive cases in Amsterdam than expected by this calculation, and slightly less in than expected in the second part of the second wave.

In [12]:
amsterdam_cases_per_week = amsterdam_cases_per_day.resample('W', label='left').sum()
amsterdam_cases_per_week.index = amsterdam_cases_per_week.index + to_offset("1D")
cases_per_municipality_per_week = total_cases_without_amsterdam_per_day.resample('W', label='left').sum()
cases_per_municipality_per_week.index = cases_per_municipality_per_week.index + to_offset("1D")

plot_cases_netherlands_amsterdam(cases_per_municipality_per_week, amsterdam_cases_per_week, 15000, 'week')

# TODO LOOK IF PLOT ABOVE IS BIAS TOTAL BECAUSE OF THE WEEKLY, MAYBE INTERPOLATE INSTEAD IN PLOT?

In [54]:
def plot_cases_vs_R(total_cases_without_amsterdam, amsterdam_cases, reproduction_numbers, max_y_range=None):
    amsterdam_cases_vs_R, total_cases_without_amsterdam_vs_R = process_cases_vs_R(amsterdam_cases, 
                                                                                  total_cases_without_amsterdam, 
                                                                                  reproduction_numbers)
    
    min_y_range, max_y_range, add_to_title = calculate_min_max_y_range(total_cases_without_amsterdam_vs_R, max_y_range)
    
    p = figure(plot_width=1000, plot_height=800,
               title=f"Relative growth of COVID-19 cases per week in Amsterdam vs. all other municipalities in the Netherlands {add_to_title}",
               x_axis_label="average reproduction number R", y_axis_label="Relative change of new COVID-19 cases in percentage", 
               toolbar_location=None, tools="", y_range=(min_y_range, max_y_range))
    
    p.circle(x=total_cases_without_amsterdam_vs_R.Rt_avg, 
             y=total_cases_without_amsterdam_vs_R.relative_growth,
             legend_label='Relative growth of COVID-19 cases in all other municipalities per day',
             color="#BDBDBD")
    
    p.circle(x=amsterdam_cases_vs_R.Rt_avg, 
             y=amsterdam_cases_vs_R.relative_growth,
             legend_label='Relative growth of COVID-19 cases in Amsterdam per day',
             color=COLORS[1])
    
    p.x_range.start = total_cases_without_amsterdam_vs_R.Rt_avg.min()
    p.x_range.end = total_cases_without_amsterdam_vs_R.Rt_avg.max()
    p.legend.location = 'top_left'
    
    show(p)

In [55]:
def process_cases_vs_R(amsterdam_cases, total_cases_without_amsterdam, reproduction_numbers):
    amsterdam_cases['relative_growth'] = amsterdam_cases.Total_reported.pct_change() * 100
    total_cases_without_amsterdam['relative_growth'] = total_cases_without_amsterdam.Total_reported.pct_change() * 100
    amsterdam_cases_vs_R = amsterdam_cases.join(reproduction_numbers)
    total_cases_without_amsterdam_vs_R = total_cases_without_amsterdam.join(reproduction_numbers)
        
    return amsterdam_cases_vs_R, total_cases_without_amsterdam_vs_R

In [56]:
def calculate_min_max_y_range(total_cases_without_amsterdam_vs_R, max_y_range):
    lowest_value = total_cases_without_amsterdam_vs_R.relative_growth.replace([np.inf, -np.inf], np.nan).min()
    min_y_range =  lowest_value - (lowest_value/10)
    print(min_y_range)
    
    add_to_title = ""
    if max_y_range == None:
        highest_value = total_cases_without_amsterdam_vs_R.relative_growth.replace([np.inf, -np.inf], np.nan).max()
        max_y_range = highest_value + (highest_value/10)
    else:
        add_to_title = "(outliers trimmed of)"
        
    return min_y_range, max_y_range, add_to_title

In [57]:
plot_cases_vs_R(cases_per_municipality_per_week, amsterdam_cases_per_week, R_per_day)

plot_cases_vs_R(cases_per_municipality_per_week, amsterdam_cases_per_week, R_per_day, 600)

-42.53336070622049


-42.53336070622049


# TODO: don't look at total new COVID-19 cases on Y-axis, look at relative growth instead!